In [20]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision.models as models
from PIL import Image
import json
from matplotlib.ticker import FormatStrFormatter
import json
import torchvision
import torch.nn as nn
from torchinfo import summary
from tqdm.auto import notebook_tqdm
from typing import Dict, List, Tuple
import os
import zipfile
from pathlib import Path
import requests
import gradio as gr
from timeit import default_timer as timer
import random
from PIL import Image
import os
import scipy.io
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision.datasets import ImageFolder
import timm
import torch.optim as optim
import time

device = "cuda" if torch.cuda.is_available() else "cpu"

In [21]:
data_dir = 'data/flowers-102/'
train_dir = data_dir + 'train'
test_dir = data_dir + 'test'
val_dir = data_dir + 'valid'

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [22]:
# Transforms for 3 sets of data - only augmentation to the training data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])

In [23]:
# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
val_data = datasets.ImageFolder(val_dir, transform=validation_transforms)
test_data = datasets.ImageFolder(test_dir ,transform=test_transforms)

# Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32,shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [53]:
flower_classes = train_data.classes
print(flower_classes)

['1', '10', '100', '101', '102', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


In [24]:
# Create model
flower_model = timm.create_model('efficientnet_b2', pretrained=True)

# 1408 input features
num_features = flower_model.classifier.in_features
num_classes = 102
flower_model.classifier = nn.Linear(num_features, num_classes)
flower_model.to(device)

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

In [27]:
# Define a loss function and optimization function

loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = optim.Adam(flower_model.parameters(), lr=0.001)

In [28]:
# Number of epochs
EPOCHS = 10

# Lists to keep track of training and validation metrics
train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(EPOCHS):
    start_time = time.time()
    
    # Training
    flower_model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward + backward + optimize
        outputs = flower_model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
    
    # Validation
    flower_model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = flower_model(images)
            loss = loss_fn(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # Calculate average losses and accuracy
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = 100 * correct / total
    
    # Print statistics
    print(f'Epoch {epoch+1}/{EPOCHS}')
    print(f'- Train loss: {train_loss:.4f}')
    print(f'- Validation loss: {val_loss:.4f}')
    print(f'- Validation accuracy: {val_accuracy:.2f}%')
    print(f'- Time: {time.time() - start_time:.2f}s')
    
    # Save metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

print('Training complete!')

Epoch 1/10
- Train loss: 1.4508
- Validation loss: 1.1107
- Validation accuracy: 93.77%
- Time: 76.49s
Epoch 2/10
- Train loss: 1.2258
- Validation loss: 1.0637
- Validation accuracy: 94.87%
- Time: 73.43s
Epoch 3/10
- Train loss: 1.1337
- Validation loss: 1.0383
- Validation accuracy: 95.48%
- Time: 73.13s
Epoch 4/10
- Train loss: 1.1033
- Validation loss: 0.9951
- Validation accuracy: 96.45%
- Time: 74.69s
Epoch 5/10
- Train loss: 1.0945
- Validation loss: 0.9706
- Validation accuracy: 96.33%
- Time: 73.39s
Epoch 6/10
- Train loss: 1.0517
- Validation loss: 0.9630
- Validation accuracy: 97.07%
- Time: 74.85s
Epoch 7/10
- Train loss: 1.0367
- Validation loss: 0.9430
- Validation accuracy: 97.43%
- Time: 73.48s
Epoch 8/10
- Train loss: 1.0311
- Validation loss: 0.9698
- Validation accuracy: 96.09%
- Time: 73.39s
Epoch 9/10
- Train loss: 0.9956
- Validation loss: 0.9310
- Validation accuracy: 97.19%
- Time: 74.15s
Epoch 10/10
- Train loss: 1.0126
- Validation loss: 0.9637
- Validation a

In [29]:
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

In [30]:
effnetb2_flowers_model_path = "Flowers_102.pth"
save_model(
    model=flower_model,
    target_dir="models/",
    model_name=effnetb2_flowers_model_path
)

[INFO] Saving model to: models\Flowers_102.pth


In [43]:
flowers_demo_path = Path("demos/flowers_102")
flowers_demo_path.mkdir(parents=True,
                               exist_ok=True)

(flowers_demo_path / "examples").mkdir(parents=True, exist_ok=True)

In [45]:
# Create a class_names.txt file from the JSON
sorted_class_ids = sorted(cat_to_name.keys(), key=lambda x: int(x))
sorted_class_names = [cat_to_name[class_id] for class_id in sorted_class_ids]
class_names_txt = "\n".join(sorted_class_names)

with open('demos/class_names.txt', 'w') as f:
    f.write(class_names_txt)

In [51]:
# Numeric
flowers_class_names_path = flowers_demo_path / "class_names_numeric.txt"

with open(flowers_class_names_path, "w") as f:
    f.write("\n".join(flower_classes))

In [46]:
%%writefile demos/flowers_102/model.py
import torch
import torchvision

from torch import nn


def create_effnetb2_model(num_classes:int=102, 
                          seed:int=42):
    """Creates an EfficientNetB2 feature extractor model and transforms.

    Args:
        num_classes (int, optional): number of classes in the classifier head. 
            Defaults to 3.
        seed (int, optional): random seed value. Defaults to 42.

    Returns:
        model (torch.nn.Module): EffNetB2 feature extractor model. 
        transforms (torchvision.transforms): EffNetB2 image transforms.
    """
    # Create EffNetB2 pretrained weights, transforms and model
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=weights)

    # Freeze all layers in base model
    for param in model.parameters():
        param.requires_grad = False

    # Change classifier head with random seed for reproducibility
    torch.manual_seed(seed)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes),
    )
    
    return model, transforms

Writing demos/flowers_102/model.py


In [48]:
%%writefile demos/flowers_102/app.py
### 1. Imports and class names setup ### 
import gradio as gr
import os
import torch

from model import create_effnetb2_model
from timeit import default_timer as timer
from typing import Tuple, Dict

# Setup class names
with open("class_names.txt", "r") as f: # reading them in from class_names.txt
    class_names = [food_name.strip() for food_name in  f.readlines()]
    
### 2. Model and transforms preparation ###    

# Create model
effnetb2, effnetb2_transforms = create_effnetb2_model(
    num_classes=102, # could also use len(class_names)
)

# Load saved weights
effnetb2.load_state_dict(
    torch.load(
        f="Flowers_102.pth",
        map_location=torch.device("cpu"),  # load to CPU
    )
)

### 3. Predict function ###

# Create predict function
def predict(img) -> Tuple[Dict, float]:
    """Transforms and performs a prediction on img and returns prediction and time taken.
    """
    # Start the timer
    start_time = timer()
    
    # Transform the target image and add a batch dimension
    img = effnetb2_transforms(img).unsqueeze(0)
    
    # Put model into evaluation mode and turn on inference mode
    effnetb2.eval()
    with torch.inference_mode():
        # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
        pred_probs = torch.softmax(effnetb2(img), dim=1)
    
    # Create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    
    # Calculate the prediction time
    pred_time = round(timer() - start_time, 5)
    
    # Return the prediction dictionary and prediction time 
    return pred_labels_and_probs, pred_time

### 4. Gradio app ###

# Create title, description and article strings
title = "Nature Walk Flower Classifier"
description = "An EfficientNetB2 feature extractor computer vision model to classify images of flowers."
article = "Created by Myles Penner (https://github.com/MylesJP)"

# Create examples list from "examples/" directory
example_list = [["examples/" + example] for example in os.listdir("examples")]

# Create Gradio interface 
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Label(num_top_classes=5, label="Predictions"),
        gr.Number(label="Prediction time (s)"),
    ],
    examples=example_list,
    title=title,
    description=description,
    article=article,
)

# Launch to Gradio
demo.launch()

Overwriting demos/flowers_102/app.py


In [50]:
%%writefile demos/flowers_102/requirements.txt
torch==1.12.0
torchvision==0.13.0
gradio==3.1.4

Writing demos/flowers_102/requirements.txt
